#Make into module, function specify annotator, directory as args
#URG: updae tile_undone_npy

#!/usr/bin/env python
# coding: utf-8
import numpy as np
import pandas as pd
import os
import os.path
import urllib.request
import progressbar # pip install progressbar2, not progressbar

import os
import shutil

import argparse

import tempfile
import urllib
import shutil
import os
import os.path
import sys

import PIL
from PIL import Image

import math
import numpy as np
import pandas as pd
import rtree
import pickle

import progressbar # pip install progressbar2, not progressbar

from geopy.geocoders import Nominatim

from contextlib import suppress

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch

import warnings
from zipfile import ZipFile

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

In [11]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
path_positive_images_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"chips_positive")
path_to_verified_sets = os.path.join(parent_directory, path_to_complete_dataset,"verified//verified_sets")

In [5]:
dist = ap.annotator("test") #create the processing class
dist.state_dcc_directory("C:\chip_allocation")

dist.number_of_tiles(2)

dist.get_tile_urls("temp_tile_name_tile_url.npy")

dist.make_subdirectories()
dist.download_images()
dist.tile_rename()
dist.chip_tiles()


Unlabeled Tiles (2, 2)
0
Bypassing download of already-downloaded file m_3208164_ne_17_060_20190829.tif
1
Bypassing download of already-downloaded file m_3208164_ne_17_060_20191109.tif
['https_naipeuwest.blob.core.windows.net_naip_v002_ga_2019_ga_60cm_2019_32081_m_3208164_ne_17_060_20191109.tif', 'https_naipeuwest.blob.core.windows.net_naip_v002_sc_2019_sc_60cm_2019_32081_m_3208164_ne_17_060_20190829.tif']
chip tiles
ga_60cm_2019_32081_m_3208164_ne_17_060_20191109.tif
505
sc_60cm_2019_32081_m_3208164_ne_17_060_20190829.tif
551


In [46]:
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(parent_directory)
unique_positive_jpgs

(23531, 2)


array([['al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000126.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000126.jpg'],
       ['al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000444.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000444.jpg'],
       ['al_60cm_2019_30088_m_3008816_se_16_060_20191115_000051.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008816_se_16_060_20191115_000051.jpg'],
       ...,
       ['wy_60cm_2019_44107_m_4410749_se_13_060_20190828_000460.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\wy_60cm_2019_44107_m_4410749_se_13_060_20190828_000460.jpg'],
       ['wy_60cm_2019_44108_m_4410801_ne_12_060_20190821_000041.jpg',
        '/

In [45]:
u, c = np.unique(unique_positive_jpgs[:,1], return_counts=True)
dup = u[c > 1]
len(dup)
#downloaded_tile

0

In [32]:
#Determine the corresponding tile names (formatted) for given paths of jpgs 
print(len(unique_positive_jpgs))
tile_names_formatted_storage_space = np.unique(fc.jpg_path_to_tile_name_formatted(unique_positive_jpgs[:,1]))
print(len(tile_names_formatted_storage_space))

23531
2077


In [33]:
#Deteremine the tile names (standard) for given paths of jpgs 
tile_names_standard_storage_space = []
for tile_name_formatted_storage_space in tile_names_formatted_storage_space:
    tile_names_standard_storage_space.append(tile_name_formatted_storage_space.split("_",4)[4])
    
#Create a 2d array of the standard and formatted tile names of the jpgs given
tile_names_in_storage_space_standard_formatted = np.hstack((np.array(tile_names_standard_storage_space)[:,None], tile_names_formatted_storage_space[:,None]))

In [55]:
u, c = np.unique(tile_names_in_storage_space_standard_formatted[:,0], return_counts=True)
dup = u[c > 1]
len(dup)
#downloaded_tile

np.sort(tile_names_in_storage_space_standard_formatted[np.isin(tile_names_in_storage_space_standard_formatted[:,0],dup)])

array(['m_3311822_ne_11_060_20180723', 'm_3708701_se_16_060_20180905',
       'm_3708701_sw_16_060_20180905', 'm_3708704_nw_16_060_20180725',
       'm_3708704_sw_16_060_20180725', 'm_3708711_ne_16_060_20180725',
       'm_3708712_nw_16_060_20180725', 'm_3708716_sw_16_060_20180929',
       'm_3712205_ne_10_060_20180723', 'm_3712205_se_10_060_20180723',
       'm_3712206_sw_10_060_20180723', 'm_3807730_se_18_060_20181025',
       'm_3808542_sw_16_060_20180727', 'm_3808543_ne_16_060_20180904',
       'm_3808549_se_16_060_20180727', 'm_3808550_nw_16_060_20180727',
       'm_3812262_se_10_060_20180723', 'm_3812263_sw_10_060_20180723',
       'm_3812264_se_10_060_20180715', 'm_3812264_sw_10_060_20180715',
       'm_4007423_se_18_060_20190830', 'm_4007423_sw_18_060_20190917',
       'm_4007424_sw_18_060_20190830', 'm_4007430_se_18_060_20190830',
       'm_4007431_nw_18_060_20190917', 'm_4007431_sw_18_060_20190917',
       'm_4009129_ne_15_060_20190708', 'm_4107114_nw_19_060_20180827',
      

In [40]:
u, c = np.unique(tile_names_in_storage_space_standard_formatted[:,1], return_counts=True)
dup = u[c > 1]
len(dup)

0

In [47]:
tile_names_in_storage_space_standard_formatted[:,1]

array(['al_60cm_2019_30088_m_3008808_sw_16_060_20191115',
       'al_60cm_2019_30088_m_3008816_se_16_060_20191115',
       'al_60cm_2019_30088_m_3008816_sw_16_060_20191115', ...,
       'wy_60cm_2019_44106_m_4410617_se_13_060_20190829',
       'wy_60cm_2019_44107_m_4410749_se_13_060_20190828',
       'wy_60cm_2019_44108_m_4410801_ne_12_060_20190821'], dtype='<U48')